In [6]:
import csv
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [23]:
#u novi folder prebacuje slike za koje postoje anotacije u csv fajlu
filename = "mpii_human_pose.csv"

fields = []
rows = []
 
with open(filename, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
 
    fields = next(csvreader)
 
    for row in csvreader:
        rows.append(row)
 
for row in rows[:len(rows)]:
    name = row[1]
    if os.path.exists("images/"+name):
        os.replace("images/"+name, "images_extract/"+name)


In [6]:
#ucitava inforacije o slikama koje sadrze vise od jedne osobe
file1 = open("images.txt","r")
b = []
lines = file1.readlines()
for i in range(len(lines)):
    lines[i] = lines[i].rstrip()
    #brise slike
    if os.path.exists("images_extract/" + i):
        os.remove( "images_extract/" + i)


In [8]:
# csv fajl cita kao pandas dataframe
df = pd.read_csv(filename)

In [11]:
#izbacuje sve redove iz tabele koji sadrze indeks slike koja se nalazi u fajlu images.txt
df = df.drop(df[df['NAME'].isin(lines)].index)

In [12]:
df

,ID,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
4,5,060111501.jpg,980,322,896,318,865,248,943,226,...,147,923,123,995,163,961,223,3.806403,curling,sports
6,7,002058449.jpg,918,456,659,518,713,413,979,288,...,270,877,247,1112,384,1012,489,8.280870,curling,sports
13,14,029122914.jpg,-1,-1,301,473,302,346,362,345,...,220,371,213,396,309,393,290,2.204083,"truck driving, loading and unloading truck, ty...",occupation
14,15,061185289.jpg,-1,-1,-1,-1,-1,-1,-1,-1,...,288,552,355,-1,-1,-1,-1,4.143112,"truck driving, loading and unloading truck, ty...",occupation
15,16,013949386.jpg,-1,-1,-1,-1,110,385,208,355,...,303,212,153,319,123,376,39,4.431105,"truck driving, loading and unloading truck, ty...",occupation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17366,17367,050240222.jpg,607,689,599,542,605,416,691,410,...,232,663,228,623,289,555,333,3.480000,pushing car,transportation
17367,17368,033474347.jpg,876,461,815,380,779,296,742,303,...,214,663,223,674,286,631,260,2.003677,pushing car,transportation
17368,17369,082650067.jpg,1038,656,950,504,903,328,831,337,...,204,729,220,704,332,675,269,3.235820,pushing car,transportation
17369,17370,072772110.jpg,562,617,558,539,553,452,511,458,...,377,490,378,470,411,461,414,1.494952,pushing car,transportation


In [20]:
#konvertuje df u csv i cuva ga
x = df.to_csv('mpii_new.csv', index=False)

In [ ]:
#kreiranje heat mapa za AVE Pooling
path = "images_test"
for image in os.listdir(path):
    #izdvajanje podataka o slici
    img = Image.open(os.path.join(path, image))
    mapa = Image.new("1", size = [img.width, img.height], color=1)

    rezultat = df[df['NAME'] == image]
    rezultat_numpy = rezultat.to_numpy()[0]
    #generisanje heat map-e
    mapa = np.array(mapa)

    boja = 0
    sirina = 10

    for k in range(2, len(rezultat_numpy) - 3, 2):
        if rezultat_numpy[k] == -1 or rezultat_numpy[k + 1] == -1:
            continue
        x = round(rezultat_numpy[k])
        y = round(rezultat_numpy[k + 1])
        for i in range(max(0, y - sirina), min(mapa.shape[0], y + sirina)):
            for j in range(max(0, x - sirina), min(mapa.shape[1], x + sirina)):
                mapa[i, j] = boja    
    mapa = cv2.resize(mapa,(368,368))      
    # Sačuvaj novu sliku
    plt.imsave('heat_maps_test_plain\\hm' + image, mapa)